In [1]:
import os
import glob
import re
import pandas as pd

In [3]:
transit_folder_path = r'S:\Github\WF-TDM-v9x\1_Inputs\4_Transit'
model = "application/v910-strategy-testing-rtp"   # model_branch_commit

In [12]:
# clean out output folder

files = glob.glob(os.path.join('output', '*'))

for file in files:
    try:
        os.remove(file)
        print(f"Deleted: {file}")
    except Exception as e:
        print(f"Error deleting {file}: {e}")

In [13]:
def preprocess_and_parse_file(file_path, file_index):
    
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Get the filename and folder name two levels up
    filename = os.path.basename(file_path)
    transit_scenario = os.path.basename(os.path.dirname(file_path))

    # Preprocess lines to remove hard returns and spaces for lines that do not start with ';' or 'LINE'
    cleaned_lines = []
    buffer = ""
    for line in lines:
        # remove extra spaces
        line = line.replace(', ', ',')
        if line.startswith(';'):
            do = 'nothing'
        elif line.startswith('LINE'):
            if buffer:
                cleaned_lines.append(buffer)
                buffer = ""
            buffer = line.strip()
        else:
            buffer += line.strip()

    # flush buffer
    if buffer:
        cleaned_lines.append(buffer)
        buffer = ""

    #display(cleaned_lines)

    # Initialize a list to store the parsed lines and a list for speed and tf positions
    parsed_data = []
    speed_tf_positions = []

    for line in cleaned_lines:

        print (line)
        pre_n_part, n_part = re.split(r'(?=N=)', line.replace('LINE ', ''), maxsplit=1)

        if pre_n_part.endswith(','):
            pre_n_part = pre_n_part[:-1]
        
        #print (pre_n_part)


        #print (pre_n_part)
        # Create dictionaries for pre-N and N
        pre_n_dict = {}
        n_dict = {'node_list': []}

        # Split pre-N part into key-value pairs
        pre_n_attributes = pre_n_part.split(',')
        #print (pre_n_attributes)
        for attr in pre_n_attributes:
            key, value = attr.split('=')
            pre_n_dict[key.strip().lower().replace('[','').replace(']','')] = value.strip().strip('"')
            #print (key.strip())
            #print (value.strip())

        #print (n_part)

        # Split N part into key-value pairs and handle special attributes
        n_attributes = n_part.split(',')
        current_n_list = []
        current_tf = None
        current_speed = None
        current_delay = None

        #print(n_attributes)

        for attr in n_attributes:
            #print (attr)
            if attr.startswith('N='):
                current_n_list.append(int(attr[2:]))
            elif attr.startswith('SPEED='):
                current_speed = float(attr[6:])
                speed_tf_positions.append({'file_index': file_index, 'name': pre_n_dict.get('name', ''), 'type': 'speed', 'node_index': len(current_n_list), 'value': current_speed})
            elif attr.startswith('TF='):
                current_tf = float(attr[3:])
                speed_tf_positions.append({'file_index': file_index, 'name': pre_n_dict.get('name', ''), 'type': 'tf', 'node_index': len(current_n_list), 'value': current_tf})
            elif attr.startswith('DELAY='):
                current_delay = float(attr[6:])
                speed_tf_positions.append({'file_index': file_index, 'name': pre_n_dict.get('name', ''), 'type': 'delay', 'node_index': len(current_n_list), 'value': current_delay})
            else:
                current_n_list.append(int(attr))
                
        if current_n_list:
            n_dict['node_list'].extend(current_n_list)

        # Explode pre-N and Combine with additional metadata
        #display(pre_n_dict)

        pre_n_dict_exploded = pd.json_normalize(pre_n_dict)
        pre_n_dict_exploded['file_index'] = file_index
        pre_n_dict_exploded['model'] = model
        pre_n_dict_exploded['transit_scenario'] = transit_scenario
        pre_n_dict_exploded['filename'] = filename
        pre_n_dict_exploded['file_path'] = file_path

        # Convert the exploded DataFrame to a dictionary and flatten it
        exploded_dict = pre_n_dict_exploded.to_dict(orient='records')[0]

        # Add the flattened dictionary to parsed_data
        exploded_dict['node_list'] = n_dict['node_list']
        parsed_data.append(exploded_dict)

    return parsed_data, speed_tf_positions

In [14]:
def read_all_lin_files(transit_folder_path):
    # Get a list of all .lin files in the folder and its subfolders
    lin_files = glob.glob(os.path.join(transit_folder_path, '**', '*.lin'), recursive=True)

    # Initialize a list to store the parsed data from all files and speed/tf positions
    all_parsed_data = []
    all_speed_tf_positions = []

    # Preprocess and parse each file, then append the data to the lists
    for file_index, file_path in enumerate(lin_files):

        print(file_path)
        file_data, speed_tf_positions = preprocess_and_parse_file(file_path, file_index)
        all_parsed_data.extend(file_data)
        all_speed_tf_positions.extend(speed_tf_positions)

    return all_parsed_data, all_speed_tf_positions

In [15]:
# Read and parse all .lin files in the folder
parsed_data, speed_tf_positions = read_all_lin_files(transit_folder_path)

# Convert the parsed data to DataFrames
lines_df = pd.DataFrame(parsed_data)

lines_df['oneway'] = lines_df['oneway'].replace({'T': True, 'F': False})
speed_tf_df = pd.DataFrame(speed_tf_positions)

files_df = lines_df.groupby(['file_index','model','transit_scenario','filename','file_path'], as_index=False).agg(num_routes=('name','count'))

lines_df = lines_df[['name','color','mode','oneway','headway1','headway2','file_index','node_list']]
#lines_df = lines_df[['name','longname','color','mode','oneway','headway1','headway2','faresystem','file_index','node_list']]

display(files_df)
display(lines_df)
display(speed_tf_df)

S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_2019\mag_brt_2019.lin
LINE NAME="UVX",COLOR=3,MODE=9,ONEWAY=F,HEADWAY[1]=6,HEADWAY[2]=10,FARESYSTEM=4,N=66744,-66743,-66750,-66768,66770,-71485,-66769,-66771,-66775,66788,-66793,-66810,-66812,-66818,-66828,-66826,-66841,50024,-66841,-66826,-66825,-66837,-66874,66882,-66893,-71504,-66922,66941,-66966,-66985,67000,-67003,-67023,67041,-67061,-71553,-71555,-67060,67059,-67058,-67057,TF=1,N=-67066,67092,SPEED=13,N=-67090,-67091,TF=1,N=-67110,-67121,-67147,-67191,67190,-67187,-67171,-67169,67172,-67174,-67173,-67175,SPEED=21,N=-67180,-67211,67255,-67284,-67292,-67299,-67328,-67357,-67366,SPEED=18,N=67371,-67373,-67374,-67375,-67377,67380,-67382,67383,-67391,-67408,TF=1,N=-67409,-67413,67421,-67396,-67411,-60530,-60529,-60528,-67407,-67405,-63443,-67461,50029
S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_2019\mag_exp_2019.lin
LINE NAME="M806_EglMtn",MODE=4,COLOR=3,HEADWAY[1]=45,HEADWAY[2]=0,ONEWAY=T,N=68503,-68543,-68579,-68612,68609,-68606,-68603,-68

,file_index,model,transit_scenario,filename,file_path,num_routes
0,0,application/v910-strategy-testing-rtp,Lin_2019,mag_brt_2019.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,1
1,1,application/v910-strategy-testing-rtp,Lin_2019,mag_exp_2019.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,2
2,2,application/v910-strategy-testing-rtp,Lin_2019,mag_lcl_2019.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,12
3,3,application/v910-strategy-testing-rtp,Lin_2019,rail_2019.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,5
4,4,application/v910-strategy-testing-rtp,Lin_2019,wfrc_og_lcl_2019.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,22
...,...,...,...,...,...,...
137,137,application/v910-strategy-testing-rtp,Lin_2050_Needs_MAG,wfrc_brt_2050UF.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,4
138,138,application/v910-strategy-testing-rtp,Lin_2050_Needs_MAG,wfrc_core_2050UF.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,36
139,139,application/v910-strategy-testing-rtp,Lin_2050_Needs_MAG,wfrc_og_lcl_2050UF.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,19
140,140,application/v910-strategy-testing-rtp,Lin_2050_Needs_MAG,wfrc_sl_exp_2050UF.lin,S:\Github\WF-TDM-v9x\1_Inputs\4_Transit\Lin_20...,2


,name,color,mode,oneway,headway1,headway2,file_index,node_list
0,UVX,3,9,False,6,10,0,"[66744, -66743, -66750, -66768, 66770, -71485,..."
1,M806_EglMtn,3,4,True,45,0,1,"[68503, -68543, -68579, -68612, 68609, -68606,..."
2,M807_NoCnty,13,4,True,30,0,1,"[50040, -69611, -69598, -69589, -69585, -69563..."
3,M821_Psn,6,4,False,30,60,2,"[65403, -65402, 65401, 65400, 65399, 65398, 65..."
4,M822_Psn,6,4,False,60,0,2,"[65403, -65402, 65401, 65400, 65399, 65398, 65..."
...,...,...,...,...,...,...,...,...
1741,SF232,9,4,False,30,30,141,"[15036, -21763, -21755, -21763, 21783, -21848,..."
1742,SF514,13,4,False,30,30,141,"[10008, -21331, -21385, -21390, -21397, -21401..."
1743,SF525,7,4,False,30,30,141,"[10010, -21973, -21998, 22027, -22061, 22081, ..."
1744,SF570,13,4,False,30,30,141,"[15059, -22663, -22687, 22747, 22742, -22733, ..."


,file_index,name,type,node_index,value
0,0,UVX,tf,41,1.00
1,0,UVX,speed,43,13.00
2,0,UVX,tf,45,1.00
3,0,UVX,speed,57,21.00
4,0,UVX,speed,66,18.00
...,...,...,...,...,...
557,138,FthlDr2100,tf,33,1.00
558,138,FthlDr2100,speed,40,21.00
559,138,BRT3533S_Core,tf,1,0.75
560,138,BRT3533S_Core,tf,2,0.75


In [16]:
# create nodes dataset

# Normalize the N column and create 'stop' column
nodes_df = lines_df.explode('node_list').reset_index(drop=True)
nodes_df = nodes_df[['file_index','name', 'node_list']]
nodes_df.rename(columns={'node_list':'node_list_entry'}, inplace=True)
nodes_df['node_id'] = nodes_df['node_list_entry'].abs()
nodes_df['is_stop'] = nodes_df['node_list_entry'] > 0

# Add the index as a separate column to indicate the position of each number in the list
nodes_df['node_seq'] = nodes_df.groupby('name').cumcount()

# Add the index as a separate column to indicate the position of each number in the list
nodes_df['node_seq'] = nodes_df.groupby('name').cumcount()


#lines_df.drop(columns=['model','transit_scenario','filename','file_path'], inplace=True)


display(nodes_df)


,file_index,name,node_list_entry,node_id,is_stop,node_seq
0,0,UVX,66744,66744,True,0
1,0,UVX,-66743,66743,False,1
2,0,UVX,-66750,66750,False,2
3,0,UVX,-66768,66768,False,3
4,0,UVX,66770,66770,True,4
...,...,...,...,...,...,...
152410,141,SF453,-21468,21468,False,1637
152411,141,SF453,-21197,21197,False,1638
152412,141,SF453,-20924,20924,False,1639
152413,141,SF453,-20878,20878,False,1640


In [17]:
# create links dataset

links_df = nodes_df.copy()

# Create the A and B columns
links_df['A'] = links_df['node_id']
links_df['B'] = links_df['node_id'].shift(-1)

# Drop the last row where B is NaN
links_df.dropna(subset=['B'], inplace=True)

# Reset index if needed
links_df.reset_index(drop=True, inplace=True)

# Drop the original N column if not needed
links_df.drop(columns=['node_list_entry', 'node_id', 'is_stop'], inplace=True)

links_df['direction'] = 1

#df['link_id'] = df['link_id']

links_df

,file_index,name,node_seq,A,B,direction
0,0,UVX,0,66744,66743,1
1,0,UVX,1,66743,66750,1
2,0,UVX,2,66750,66768,1
3,0,UVX,3,66768,66770,1
4,0,UVX,4,66770,71485,1
...,...,...,...,...,...,...
152409,141,SF453,1636,21551,21468,1
152410,141,SF453,1637,21468,21197,1
152411,141,SF453,1638,21197,20924,1
152412,141,SF453,1639,20924,20878,1


In [18]:
# append links in other direction for oneway==False

display(links_df)

merged_df = pd.merge(links_df, lines_df, on=['file_index', 'name'])

merged_df = merged_df[merged_df['oneway']==False]

all_reversed_rows = pd.DataFrame()

# Group by 'file_index' and 'name'
grouped = merged_df.groupby(['file_index', 'name'])

for (file_index, name), group in grouped:
    if group['oneway'].iloc[0] == False:
        # Reverse the A and B columns
        reversed_rows = group.copy()
        reversed_rows['A'], reversed_rows['B'] = reversed_rows['B'], reversed_rows['A']

        # Flip the rows based on sequence sorted in descending order
        reversed_rows = reversed_rows.sort_values(by='node_seq', ascending=False).reset_index(drop=True)

        # Find the maximum node_seq value in the original group
        max_node_seq = group['node_seq'].max()

        # Assign new node_seq values to the reversed rows
        reversed_rows['node_seq'] = range(max_node_seq + 1, max_node_seq + 1 + len(reversed_rows))

        # Assign direction = 2
        reversed_rows['direction'] = 2

        # Drop the 'oneway' column from the reversed rows if not needed
        reversed_rows.drop(columns=['oneway'], inplace=True)

        # Collect the reversed rows
        all_reversed_rows = pd.concat([all_reversed_rows,reversed_rows])

display(all_reversed_rows)

# Get the list of columns in links_df
columns_to_keep = links_df.columns

# Filter the columns in links_with_reverse_direction_df to only include those in links_df
all_reversed_rows = all_reversed_rows[columns_to_keep]

# Append these new rows to the original DataFrame
links_with_reverse_direction_df = pd.concat([links_df, all_reversed_rows])

# node sequence is now link sequence
links_with_reverse_direction_df.rename(columns={'node_seq':'link_seq'}, inplace=True)

# add link_id
links_with_reverse_direction_df['link_id'] = links_with_reverse_direction_df['A'].astype(str) + '_' + links_with_reverse_direction_df['B'].astype(str)

links_with_reverse_direction_df = links_with_reverse_direction_df.sort_values(by=['file_index','name','link_seq'], ascending=True).reset_index(drop=True)


display(links_with_reverse_direction_df)

,file_index,name,node_seq,A,B,direction
0,0,UVX,0,66744,66743,1
1,0,UVX,1,66743,66750,1
2,0,UVX,2,66750,66768,1
3,0,UVX,3,66768,66770,1
4,0,UVX,4,66770,71485,1
...,...,...,...,...,...,...
152409,141,SF453,1636,21551,21468,1
152410,141,SF453,1637,21468,21197,1
152411,141,SF453,1638,21197,20924,1
152412,141,SF453,1639,20924,20878,1


,file_index,name,node_seq,A,B,direction,color,mode,headway1,headway2,node_list
0,0,UVX,89,68503,50029,2,3,9,6,10,"[66744, -66743, -66750, -66768, 66770, -71485,..."
1,0,UVX,90,50029,67461,2,3,9,6,10,"[66744, -66743, -66750, -66768, 66770, -71485,..."
2,0,UVX,91,67461,63443,2,3,9,6,10,"[66744, -66743, -66750, -66768, 66770, -71485,..."
3,0,UVX,92,63443,67405,2,3,9,6,10,"[66744, -66743, -66750, -66768, 66770, -71485,..."
4,0,UVX,93,67405,67407,2,3,9,6,10,"[66744, -66743, -66750, -66768, 66770, -71485,..."
...,...,...,...,...,...,...,...,...,...,...,...
39,141,SF570,763,22733,22742,2,13,4,30,30,"[15059, -22663, -22687, 22747, 22742, -22733, ..."
40,141,SF570,764,22742,22747,2,13,4,30,30,"[15059, -22663, -22687, 22747, 22742, -22733, ..."
41,141,SF570,765,22747,22687,2,13,4,30,30,"[15059, -22663, -22687, 22747, 22742, -22733, ..."
42,141,SF570,766,22687,22663,2,13,4,30,30,"[15059, -22663, -22687, 22747, 22742, -22733, ..."


,file_index,name,link_seq,A,B,direction,link_id
0,0,UVX,0,66744,66743,1,66744_66743
1,0,UVX,1,66743,66750,1,66743_66750
2,0,UVX,2,66750,66768,1,66750_66768
3,0,UVX,3,66768,66770,1,66768_66770
4,0,UVX,4,66770,71485,1,66770_71485
...,...,...,...,...,...,...,...
217585,141,SF570,763,22733,22742,2,22733_22742
217586,141,SF570,764,22742,22747,2,22742_22747
217587,141,SF570,765,22747,22687,2,22747_22687
217588,141,SF570,766,22687,22663,2,22687_22663


In [10]:
# export to CSV
files_df                       .to_csv('output/files.csv'          , index=False)
lines_df                       .to_csv('output/lines.csv'          , index=False)
speed_tf_df                    .to_csv('output/speeds-tf-delay.csv', index=False)
nodes_df                       .to_csv('output/nodes.csv'          , index=False)
links_with_reverse_direction_df.to_csv('output/links.csv'          , index=False)